In [1]:
from langdetect import detect

# Assuming you have already extracted the topic names and labels from your documents
topic_names = ["PDF", "Image"]
labels = ["english_words"]  # Replace with the actual language labels

# Vectorize the text data using Word Embeddings 
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Input, Dense, Flatten
from keras.models import Model

# Create an empty list to store the text data
texts = []

# Define the path to the PDF documents folder
path_to_pdf_folder = "pdf_documents"
# Loop through each file in the folder and append the text to the list
for filename in os.listdir(path_to_pdf_folder):
    with open(os.path.join(path_to_pdf_folder, filename), "rb") as file:
        try:
            text = file.read().decode("utf-8")
        except UnicodeDecodeError:
            try:
                text = file.read().decode("latin-1")
            except UnicodeDecodeError:
                text = file.read().decode("ISO-8859-1")
        texts.append(text)

# Define the path to the Image documents folder
path_to_image_folder = "image_documents"
# Loop through each file in the folder and append the text to the list
for filename in os.listdir(path_to_image_folder):
    with open(os.path.join(path_to_image_folder, filename), "rb") as file:
        try:
            text = file.read().decode("utf-8")
        except UnicodeDecodeError:
            try:
                text = file.read().decode("latin-1")
            except UnicodeDecodeError:
                text = file.read().decode("ISO-8859-1")
        texts.append(text)

# Tokenize the text data
# Define the maximum number of words to keep in the vocabulary
max_words = 10000

# Create a tokenizer object
tokenizer = Tokenizer(num_words=max_words)

# Fit the tokenizer on the text data
tokenizer.fit_on_texts(texts)

# Convert the text data to a sequence of integers
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequences to make them of equal length
# Define the maximum length of the sequence
max_length = 1000

# Pad the sequences to make them of equal length
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Define the Word Embedding layer
# Define the size of the embedding vector
embedding_size = 100

# Define the input layer
inputs = Input(shape=(max_length,))

# Define the embedding layer
embedding = Embedding(input_dim=max_words, output_dim=embedding_size, input_length=max_length)(inputs)

# Define the model architecture
# Flatten the embedding layer
flatten = Flatten()(embedding)

# Add a Dense layer with ReLU activation and 256 neurons
dense1 = Dense(256, activation='relu')(flatten)

# Add another Dense layer with ReLU activation and 128 neurons
dense2 = Dense(128, activation='relu')(dense1)

# Define the output layer
outputs = Dense(len(topic_names), activation='softmax')(dense2)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Define the labels for the data
# Convert the language labels to numerical labels
class_mapping = {"marathi": 0, "english_words": 1, "hindi": 2}
numerical_labels = [class_mapping[label] for label in labels]

# Convert the labels to categorical format
categorical_labels = to_categorical(numerical_labels)

# Split the data into training and test sets
train_data = padded_sequences[:80]
train_labels = categorical_labels[:80]
test_data = padded_sequences[80:]
test_labels = categorical_labels[80:]

# Duplicate the samples in the arrays to match the required size
num_duplicates = len(train_data) // len(train_labels)
train_labels = np.repeat(train_labels, num_duplicates, axis=0)

# Train the model
model.fit(train_data, train_labels, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels, batch_size=32)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)


2023-07-03 22:24:02.511214: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 22:24:05.574371: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 22:24:05.581347: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 22:24:10.919989: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/20
2/2 [==============================] - 2s 379ms/step - loss: 0.5275 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
2/2 [==============================] - 0s 218ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
2/2 [==============================] - 0s 226ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
2/2 [==============================] - 0s 219ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
2/2 [==============================] - 0s 231ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
2/2 [==============================] - 0s 216ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 0s 249ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.

/home/kranthi/anaconda3/lib/python3.9/site-packages/keras/utils/generic_utils.py:238: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: cannot convert float infinity to integer